In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import sys
from datetime import datetime, date, timedelta
import os
import re
from IPython.display import clear_output

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0'}
FQDN = 'https://www.h******.**'
responce = requests.get(FQDN)

print("request denied") if responce.status_code != 200 else print("ok")

In [ ]:
def get_html(url, params=None):
    r = requests.get(url, headers=header, params=params)
    return r

In [ ]:
def flatten(L: list) -> list:
    '''рекурентное преобразование списка списков списков... в плоский одноранговый список'''
    if len(L) > sys.getrecursionlimit():
        sys.setrecursionlimit(len(L)*10)
    if L == []:
        return L
    if isinstance(L[0], list):
        return flatten(L[0]) + flatten(L[1:])
    rez = list(L[:1] + flatten(L[1:]))
    return rez

In [ ]:
def str_to_date(str_date: str) -> datetime.date:
    '''принимает дату 12.11.2017 в формате string, возвращает в формате datetime'''
    temp_date = [d for d in str_date.split('.')]
    temp_date.reverse()
    temp_date = '-'.join(temp_date)
    return date.fromisoformat(temp_date)

In [ ]:
def dates_urls_titles_on_page(page_url: str) -> list:
    '''возвращает список дат, ссылок, тайтлов на статьи со страницы'''
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    short_urls = [e.a.get('href') for e in soup.find_all('h2', class_='first_child')][::-1]
    fqdn_urls = [FQDN + elem if 'http' not in elem and elem.startswith('/') else elem for elem in short_urls]
    cur_date = '.'.join(re.findall('(\d+)', page_url)[::-1])
    dates = [cur_date] * len(short_urls)
    titles = [e.a.text for e in soup.find_all('h2', class_='first_child')][::-1]
    return dates, fqdn_urls, titles

In [ ]:
def dates_urls_titles_on_page_filial(page_url: str) -> list:
    
    '''возвращает список дат, ссылок, тайтлов на статьи со страницы'''
    dates = []
    fqdn_urls = []
    month_list = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн',
           'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    short_urls = [e.a.get('href') for e in soup.find_all('h2', class_='first_child')]
    for elem in short_urls:
        if 'http' not in elem and elem.startswith('/') and 'h******.**' not in elem:
            fqdn_urls.append(FQDN + elem)
        elif 'http' not in elem and elem.startswith('/'):
            fqdn_urls.append('https:' + elem)
        else:
            fqdn_urls.append(elem)
    
    for e in soup.find_all('div', class_='post-meta__date'):
        day = e.get_text(' ', strip=True).split()[0]
        month = str(month_list.index(e.get_text(' ', strip=True).split()[1]) + 1)
        year = e.get_text(' ', strip=True).split()[2]
        if len(month) < 2:
            month = '0' + month
        if len(day) < 2:
            day = '0' + day
        dates.append(day + '.' + month + '.' + year)
    
    titles = [e.a.text for e in soup.find_all('h2', class_='first_child')]
    return dates, fqdn_urls, titles

In [ ]:
def get_content(page_url: str):
    '''сборщик контента. возвращает 2 списка - текст, внутренние ссылки'''
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')

    text_content = [e.get_text(' ', strip=True).replace('\xa0',' ').replace('\t','').replace('\n',' ').replace('  ',' ') \
                    for e in soup.find_all('div', class_='post__text')]
    
    if len(text_content) == 0:
        for e in soup.find_all('div', class_='events__content post post_first'):
            try:
                e.find('div', class_='flex flex--gap').extract()
            except (AttributeError):
                pass
            try:
                e.find('h5').extract()
            except (AttributeError):
                pass
            try:
                e.find('div', class_='signup with-indent6').extract()
            except (AttributeError):
                pass
            try:
                e.find('form').extract()
            except (AttributeError):
                pass
            
            text_content.append(e.get_text(' ', strip=True).replace('\xa0',' ').replace('\t','').replace('\n',' ').replace('  ',' '))
      
    if len(text_content) == 0:
        text_content = [e.get_text(' ', strip=True).replace('\xa0',' ').replace('\t','').replace('\n',' ').replace('  ',' ') \
                for e in soup.find_all('div', class_='b-row')]

    if len(text_content) == 0:
        text_content = [e.get_text(' ', strip=True).replace('\xa0',' ').replace('\t','').replace('\n',' ').replace('  ',' ') \
                for e in soup.find_all('div', class_='block-news-single__content')]

    if len(text_content) == 0:
        text_content = [e.get_text(' ', strip=True).replace('\xa0',' ').replace('\t','').replace('\n',' ').replace('  ',' ') \
                for e in soup.find_all('div', class_='container-fluid')]
        
    
    urls_content = [[FQDN + i.get('href') if 'http' not in i.get('href') and 'h*****.**' not in i.get('href') \
                     else i.get('href') for i in e.find_all('a', href=True)] \
                    for e in soup.find_all('div', class_='post__text')]
    
    if len(urls_content) == 0:
        for e in soup.find_all('div', class_='events__content post post_first'):
            try:
                e.find('div', class_='flex flex--gap').extract()
            except (AttributeError):
                pass
            try:
                e.find('h5').extract()
            except (AttributeError):
                pass
            try:
                e.find('div', class_='signup with-indent6').extract()
            except (AttributeError):
                pass
            try:
                e.find('form').extract()
            except (AttributeError):
                pass
            
            urls_content = [FQDN + i.get('href') if 'http' not in i.get('href') and 'h******.**' not in i.get('href') \
                            else i.get('href') for i in e.find_all('a', href=True)]
            
    if len(urls_content) == 0:        
        urls_content = [[FQDN + i.get('href') if 'http' not in i.get('href') and 'h******.**' not in i.get('href') \
                     else i.get('href') for i in e.find_all('a', href=True)] \
                    for e in soup.find_all('div', class_='b-row')]
        
    if len(urls_content) == 0:        
        urls_content = [[FQDN + i.get('href') if 'http' not in i.get('href') and 'h*******.**' not in i.get('href') \
                     else i.get('href') for i in e.find_all('a', href=True)] \
                    for e in soup.find_all('div', class_='block-news-single__content')]        

    if len(urls_content) == 0:        
        urls_content = [[FQDN + i.get('href') if 'http' not in i.get('href') and 'h******.**' not in i.get('href') \
                     else i.get('href') for i in e.find_all('a', href=True)] \
                    for e in soup.find_all('div', class_='container-fluid')]   
    
    
    if len(urls_content) != 0 and isinstance(urls_content[0], list):
        return text_content, [url for url in urls_content[0] if 'jpg' not in url.lower()]
    else:
        return text_content, [url for url in urls_content if 'jpg' not in url.lower()]


In [ ]:
def flatten_column(df: pd.core.frame.DataFrame, column: str) -> pd.core.frame.DataFrame:
    '''извлекает значения в ячейках датафрейма из списочного вида в текстовый'''

    for idx in range(df.shape[0]):

        if type(df.loc[idx,column]) is list:
            df.loc[idx,column] = ', '.join(df.loc[idx,column]) if len(df.loc[idx,column]) > 0 else ''

    return df

In [ ]:
def read_conf(fqdn: str, start_id: int = 0, start_date: str = '2022-01-01'):
    file_name = 'aggregator.ini'
    domain = '.'.join(fqdn.split('/')[2].split('.')[-2:])
    start_date = date.fromisoformat(start_date)
    if not os.path.exists(file_name):
        open(file_name, 'x')
    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if domain in row[0]:
                try:
                    start_date = date.fromisoformat(row[1])
                except (ValueError):
                    start_date = start_date
                try:
                    start_id = abs(int(float(row[2])))
                except (ValueError):
                    start_id = start_id
                    
    return start_date, start_id

In [ ]:
def write_conf(fqdn: str, end_id: int, end_date: str = str(datetime.now().date())):
    domain = '.'.join(fqdn.split('/')[2].split('.')[-2:])
    file_name = 'aggregator.ini'
    fin = open(file_name, "rt")
    data = fin.read()
    domain_list = []
    for elem in data.split('\n'):
        domain_list.append(elem.split(',')[0])
        if elem.split(',')[0] == domain:
            data = data.replace(domain + ',' + elem.split(',')[1] +',' + elem.split(',')[2], 
                                domain + ',' + end_date + ',' + str(end_id))
            break
    
    new_row = domain + ',' + end_date + ',' + str(end_id)
    if domain not in domain_list:
        data += new_row if len(domain_list[0]) == 0 else '\n' + new_row

    fin.close()
    fin = open(file_name, "wt")
    fin.write(data)
    fin.close()

In [ ]:
start_date, start_id  = read_conf(FQDN, start_id = 0, start_date = '2022-01-01') #id объявления
print(f'start_date = {start_date}\nstart_id = {start_id}')

---

###### Раздел "Город"

In [ ]:
base_url = 'https://www.h*****.**/news/archive/'
start_page = base_url + '/'.join(str(start_date).split('-'))
print(f'start_page = {start_page}')

In [ ]:
count = 0
dates = []
urls = []
titles = []
current_page = start_page

while datetime.now().date() >= start_date + timedelta(days = count):
    responce = requests.get(current_page)
    count += 1
    if responce.status_code == 200:
        dates.extend(dates_urls_titles_on_page(current_page)[0])
        urls.extend(dates_urls_titles_on_page(current_page)[1])
        titles.extend(dates_urls_titles_on_page(current_page)[2])
        current_page = base_url + '/'.join(str(start_date + timedelta(days = count)).split('-'))
    else:
        current_page = base_url + '/'.join(str(start_date + timedelta(days = count)).split('-'))
titles = titles[::-1]
dates = dates[::-1]
urls = urls[::-1]

# Количество новых статей с даты последнего запуска агрегатора
news_sum = sum([True for date_news in dates if str_to_date(date_news) > start_date])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} статей.')

In [ ]:
content = []
ext_urls = []

for page_url in urls[:news_sum]:
    print(page_url)
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    content.append(get_content(page_url)[0])
    ext_urls.append(get_content(page_url)[1])
    clear_output(wait=True)
    print(len(dates[:news_sum]),len(urls[:news_sum]),len(titles[:news_sum]),len(content),len(ext_urls))

In [ ]:
df = pd.DataFrame()
df['Дата'] = dates[:news_sum]
df['Раздел'] = '*******'
df['Заголовок'] = titles[:news_sum]
df['Текст'] = content
df['Внешние ссылки'] = ext_urls
df['URL статьи'] = urls[:news_sum]

# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df)[3:5]:
    df = flatten_column(df,column)
df.head(3)

###### Раздел "Город2"

In [ ]:
base_url = 'https://****.h*******.**/news/page'
start_page = base_url + str(1) + '.html'
print(f'start_page = {start_page}')

In [ ]:
dates = []
urls = []
titles = []
dates.extend(dates_urls_titles_on_page_filial(start_page)[0])
urls.extend(dates_urls_titles_on_page_filial(start_page)[1])
titles.extend(dates_urls_titles_on_page_filial(start_page)[2])

count=1
while str_to_date(dates[-1]) > start_date:
    count += 1
    next_page = base_url + str(count) + '.html'
    dates.extend(dates_urls_titles_on_page_filial(next_page)[0])
    urls.extend(dates_urls_titles_on_page_filial(next_page)[1])
    titles.extend(dates_urls_titles_on_page_filial(next_page)[2])

# Количество новых статей с даты последнего запуска агрегатора
news_sum = sum([True for date_news in dates if str_to_date(date_news) > start_date])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} статей.')

In [ ]:
content = []
ext_urls = []

for page_url in urls[:news_sum]:
    print(page_url)
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    content.append(get_content(page_url)[0])
    ext_urls.append(get_content(page_url)[1])
    clear_output(wait=True)
    print(len(dates[:news_sum]),len(urls[:news_sum]),len(titles[:news_sum]),len(content),len(ext_urls))

In [ ]:
df2 = pd.DataFrame()
df2['Дата'] = dates[:news_sum]
df2['Раздел'] = '********'
df2['Заголовок'] = titles[:news_sum]
df2['Текст'] = content
df2['Внешние ссылки'] = ext_urls
df2['URL статьи'] = urls[:news_sum]

In [ ]:
# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df2)[3:5]:
    df2 = flatten_column(df2,column)
df2.head(3)

###### Раздел "Город3"

In [ ]:
base_url = 'https://***.h******.**/news/page'
start_page = base_url + str(1) + '.html'
print(f'start_page = {start_page}')

In [ ]:
dates = []
urls = []
titles = []
dates.extend(dates_urls_titles_on_page_filial(start_page)[0])
urls.extend(dates_urls_titles_on_page_filial(start_page)[1])
titles.extend(dates_urls_titles_on_page_filial(start_page)[2])

count=1
while str_to_date(dates[-1]) > start_date:
    count += 1
    next_page = base_url + str(count) + '.html'
    dates.extend(dates_urls_titles_on_page_filial(next_page)[0])
    urls.extend(dates_urls_titles_on_page_filial(next_page)[1])
    titles.extend(dates_urls_titles_on_page_filial(next_page)[2])

# Количество новых статей с даты последнего запуска агрегатора
news_sum = sum([True for date_news in dates if str_to_date(date_news) > start_date])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} статей.')

In [ ]:
content = []
ext_urls = []

for page_url in urls[:news_sum]:
    print(page_url)
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    content.append(get_content(page_url)[0])
    ext_urls.append(get_content(page_url)[1])
    clear_output(wait=True)
    print(len(dates[:news_sum]),len(urls[:news_sum]),len(titles[:news_sum]),len(content),len(ext_urls))

In [ ]:
df3 = pd.DataFrame()
df3['Дата'] = dates[:news_sum]
df3['Раздел'] = '******'
df3['Заголовок'] = titles[:news_sum]
df3['Текст'] = content
df3['Внешние ссылки'] = ext_urls
df3['URL статьи'] = urls[:news_sum]

# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df3)[3:5]:
    df3 = flatten_column(df3,column)
df3.head(3)

###### Раздел "Город4"

In [ ]:
base_url = 'https://****.h*******.**/news/page'
start_page = base_url + str(1) + '.html'
print(f'start_page = {start_page}')

In [ ]:
dates = []
urls = []
titles = []
dates.extend(dates_urls_titles_on_page_filial(start_page)[0])
urls.extend(dates_urls_titles_on_page_filial(start_page)[1])
titles.extend(dates_urls_titles_on_page_filial(start_page)[2])

count=1
while str_to_date(dates[-1]) > start_date:
    count += 1
    next_page = base_url + str(count) + '.html'
    dates.extend(dates_urls_titles_on_page_filial(next_page)[0])
    urls.extend(dates_urls_titles_on_page_filial(next_page)[1])
    titles.extend(dates_urls_titles_on_page_filial(next_page)[2])

# Количество новых статей с даты последнего запуска агрегатора
news_sum = sum([True for date_news in dates if str_to_date(date_news) > start_date])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} статей.')

In [ ]:
content = []
ext_urls = []

for page_url in urls[:news_sum]:
    print(page_url)
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    content.append(get_content(page_url)[0])
    ext_urls.append(get_content(page_url)[1])
    clear_output(wait=True)
    print(len(dates[:news_sum]),len(urls[:news_sum]),len(titles[:news_sum]),len(content),len(ext_urls))

In [ ]:
df4 = pd.DataFrame()
df4['Дата'] = dates[:news_sum]
df4['Раздел'] = '*****'
df4['Заголовок'] = titles[:news_sum]
df4['Текст'] = content
df4['Внешние ссылки'] = ext_urls
df4['URL статьи'] = urls[:news_sum]

# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df4)[3:5]:
    df4 = flatten_column(df4,column)
df4.head(3)

In [ ]:
file_name = base_url.split('/')[2].split('.')[1] + '_' + str(datetime.now().date()) + '.xlsx'
with pd.ExcelWriter(file_name) as writer:
    df.to_excel(writer, sheet_name='*****', index=False)
    df2.to_excel(writer, sheet_name='******', index=False)
    df3.to_excel(writer, sheet_name='*******', index=False)
    df4.to_excel(writer, sheet_name='******', index=False)

In [ ]:
#обновление хранимого id-объявлений и даты в файле aggregator.ini
write_conf(FQDN, 0)

---